In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import json

In [2]:
data = pd.read_csv("data/loan_data.csv")
data.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [5]:
load_dotenv()
client = genai.Client()

In [6]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)
    

models/gemini-1.5-flash-001-tuning


# Training on Loan Dataset

## Preparing Training Data

In [3]:
training_data = []

In [7]:
for i in range(data.shape[0]):
    trainin_data_i=[]
    prompt=f"A {data.loc[i,"person_age"]}-year-old {data.loc[i,'person_gender']} with a {data.loc[i,'person_education']} level of education, earning ${int(data.loc[i,'person_income'])}, with {data.loc[i,'person_emp_exp']} years of work experience, who has {data.loc[i,'person_home_ownership'].lower()} housing and is applying for a ${int(data.loc[i,'loan_amnt'])}. {data.loc[i,'loan_intent'].lower()} loan at an interest rate of {data.loc[i,'loan_int_rate']}%. Their credit score is {data.loc[i,'credit_score']}, and they have a credit history of {int(data.loc[i,'cb_person_cred_hist_length'])} years. Previous defaults on file: {data.loc[i,'previous_loan_defaults_on_file']}. Should the loan be approved?"
    response = "yes" if data.loc[i, "loan_status"] == 1 else "No"

    training_data.append([prompt, response])

In [9]:
print(training_data[:10])

[['A 22.0-year-old female with a Master level of education, earning $71948, with 0 years of work experience, who has rent housing and is applying for a $35000. personal loan at an interest rate of 16.02%. Their credit score is 561, and they have a credit history of 3 years. Previous defaults on file: No. Should the loan be approved?', 'yes'], ['A 21.0-year-old female with a High School level of education, earning $12282, with 0 years of work experience, who has own housing and is applying for a $1000. education loan at an interest rate of 11.14%. Their credit score is 504, and they have a credit history of 2 years. Previous defaults on file: Yes. Should the loan be approved?', 'No'], ['A 25.0-year-old female with a High School level of education, earning $12438, with 3 years of work experience, who has mortgage housing and is applying for a $5500. medical loan at an interest rate of 12.87%. Their credit score is 635, and they have a credit history of 3 years. Previous defaults on file:

## Fine Tuning

In [8]:
training_dataset = types.TuningDataset(
    examples=[
        types.TuningExample(
            text_input=i,
            output=o,
        )
        for i,o in training_data
    ]
)

In [73]:
tuning_job = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset,
    config=types.CreateTuningJobConfig(
        epoch_count=5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="gem-2.0-flash-LoanApproval"
    )
)

tuning_job

TuningJob(name='tunedModels/gem20flashloanapproval-g6wazf6qmmbz', state=<JobState.JOB_STATE_QUEUED: 'JOB_STATE_QUEUED'>, create_time=None, start_time=None, end_time=None, update_time=None, error=None, description=None, base_model=None, tuned_model=None, supervised_tuning_spec=None, tuning_data_stats=None, encryption_spec=None, partner_model_tuning_spec=None, distillation_spec=None, experiment=None, labels=None, pipeline_job=None, tuned_model_display_name=None)

In [9]:
tuning_job.state

NameError: name 'tuning_job' is not defined

In [24]:
response = client.models.generate_content(
    model="tunedModels/gem20flashloanapproval-uoamepmbr22k",
    contents=str(training_data[0][0])
)


In [23]:
print(response.text)

None


In [20]:
client.tunings.get(name="tunedModels/gem20flashloanapproval-9j70uinjcu3g")

TuningJob(name='tunedModels/gem20flashloanapproval-9j70uinjcu3g', state=<JobState.JOB_STATE_RUNNING: 'JOB_STATE_RUNNING'>, create_time=datetime.datetime(2025, 4, 20, 8, 16, 36, 144315, tzinfo=TzInfo(UTC)), start_time=None, end_time=None, update_time=datetime.datetime(2025, 4, 20, 8, 16, 36, 144315, tzinfo=TzInfo(UTC)), error=None, description=None, base_model='models/gemini-1.5-flash-001-tuning', tuned_model=TunedModel(model='tunedModels/gem20flashloanapproval-9j70uinjcu3g', endpoint='tunedModels/gem20flashloanapproval-9j70uinjcu3g'), supervised_tuning_spec=None, tuning_data_stats=None, encryption_spec=None, partner_model_tuning_spec=None, distillation_spec=None, experiment=None, labels=None, pipeline_job=None, tuned_model_display_name=None)

In [11]:
for i, tuned_model in enumerate(client.tunings.list()):
    print(f"{i+1} -> {tuned_model.name} state: {tuned_model.state}\n")
    

1 -> tunedModels/gem20flashloanapproval-uoamepmbr22k state: JobState.JOB_STATE_SUCCEEDED

2 -> tunedModels/gem20flashloanapproval-2pl9k5nx5l8g state: JobState.JOB_STATE_RUNNING

3 -> tunedModels/gem20flashloanapproval-yz2tjlu62jhb state: JobState.JOB_STATE_RUNNING

4 -> tunedModels/gem20flashloanapproval-n7ckykl4vhrk state: JobState.JOB_STATE_RUNNING

5 -> tunedModels/gem20flashloanapproval-9j70uinjcu3g state: JobState.JOB_STATE_RUNNING

6 -> tunedModels/gem20flashloanapproval-g6wazf6qmmbz state: JobState.JOB_STATE_RUNNING



In [12]:
client.tunings.(name="tunedModels/gem20flashloanapproval-uoamepmbr22k")

SyntaxError: invalid syntax (3412801192.py, line 1)

# Training for a Company Support Bot

## Preparing data

In [16]:
with open("data/cmp_faq_100.json","r") as faq_json:
    faq_data = json.load(faq_json)

In [18]:
faq_data[:10]

[{'prompt': 'What services does TechNova Solutions offer?',
  'output': 'TechNova Solutions offers a comprehensive range of services including Web & Mobile App Development, Cloud Infrastructure & DevOps, Cybersecurity, Data Analytics & AI, IoT & Embedded Systems, and IT Consulting & Training.'},
 {'prompt': 'Can TechNova Solutions develop a custom mobile app for my business?',
  'output': 'Yes, TechNova Solutions specializes in custom Web & Mobile App Development, creating tailored solutions for various business needs.'},
 {'prompt': 'Do you provide cloud migration services?',
  'output': 'Absolutely. Our Cloud Infrastructure & DevOps team assists with seamless cloud migration, optimization, and management.'},
 {'prompt': 'What kind of cybersecurity services do you offer?',
  'output': 'Our Cybersecurity services include vulnerability assessments, penetration testing, security audits, incident response planning, and implementing robust security solutions to protect your digital assets.

## Training 

In [35]:
training_dataset_faq = types.TuningDataset(
    examples=[
        types.TuningExample(
            text_input=data["prompt"],
            output=data["output"],
        )
        for data in faq_data
    ]
)

In [36]:
tuning_job_faq = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset_faq,
    config=types.CreateTuningJobConfig(
        epoch_count=5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="faq_tuned_model"
    )
)

/tmp/ipykernel_8315/2496635928.py:1: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_job_faq = client.tunings.tune(


In [12]:
for i, tuned_model in enumerate(client.tunings.list()):
    print(f"{i+1} -> {tuned_model.name} state: {tuned_model.state}\n")

1 -> tunedModels/gem20flashloanapproval-uoamepmbr22k state: JobState.JOB_STATE_SUCCEEDED

2 -> tunedModels/gem20flashloanapproval-2pl9k5nx5l8g state: JobState.JOB_STATE_SUCCEEDED

3 -> tunedModels/gem20flashloanapproval-yz2tjlu62jhb state: JobState.JOB_STATE_SUCCEEDED

4 -> tunedModels/gem20flashloanapproval-n7ckykl4vhrk state: JobState.JOB_STATE_SUCCEEDED

5 -> tunedModels/gem20flashloanapproval-9j70uinjcu3g state: JobState.JOB_STATE_SUCCEEDED

6 -> tunedModels/gem20flashloanapproval-g6wazf6qmmbz state: JobState.JOB_STATE_SUCCEEDED

7 -> tunedModels/faqtunedmodel-2mi1ma079yipeb6tb1979ho6sz state: JobState.JOB_STATE_SUCCEEDED



## Testing Responses

In [7]:
def gen_reponse(prompt):
    client = genai.Client(api_key=os.getenv("gem_api"))
    response = client.models.generate_content(
        model="tunedModels/faqtunedmodel-2mi1ma079yipeb6tb1979ho6sz", contents=prompt
    )
    
    print(response.text)

In [8]:
gen_reponse("What is the usual cost for building website in Technova?")

We don't provide pricing for specific services like website development without first understanding project requirements and scope. Contact us for a free consultation where we can discuss your project and provide an estimated range.


In [8]:
gen_reponse("What solutions does Technova Provide?")

Technova Solutions & Services provides a comprehensive range of solutions, including Web & Mobile App Development, Cloud Infrastructure & DevOps, Data Analytics & AI, Cybersecurity, IoT & Embedded Systems, and IT Consulting & Training.


In [10]:
gen_reponse("What tech stack does Technova Uses?")

Technova uses a comprehensive tech stack, adapting to project needs, encompassing front-end frameworks, back-end languages, cloud platforms, DevOps tools, and data analytics technologies.
